In [ ]:
import numpy as np
import scipy
import random
import urllib
import matplotlib.pyplot as plt
import json

In [ ]:
random.seed(0) #set an initial seed so that we can reproduce consistently

In [ ]:
def readJSONfromFile(fname):
    yelp_data = []
    with open(fname) as f:
        for line in f:
            yelp_data.append(json.loads(line))
    return yelp_data

In [ ]:
IuVegas = readJSONfromFile('yelp_user_IuVegas.json')
UiVegas = readJSONfromFile('yelp_user_UiVegas.json')

In [ ]:
IuVegas = IuVegas[0]
UiVegas = UiVegas[0]

In [ ]:
I = readJSONfromFile('yelp_restaurant_data.json')

In [ ]:
I = I[0]

In [ ]:
#I[:10]

In [ ]:
VegasUser = []
for user in IuVegas.keys():
    if len(IuVegas[user]) >= 10:
        VegasUser.append(user)

In [ ]:
len(IuVegas[VegasUser[0]])


In [ ]:
len(IuVegas)

In [ ]:
S_ui_test = {}

In [ ]:
for user in VegasUser:
    temp = random.choice(IuVegas[user])
    IuVegas[user].remove(temp)
    S_ui_test[user] = temp

In [ ]:
len(IuVegas[VegasUser[0]])

In [ ]:
len(S_ui_test)

In [ ]:
len(VegasUser)

In [ ]:
VegasRest = UiVegas.keys()

In [ ]:
len(VegasUser)

In [ ]:
len_u = []
len_i = []
for k in IuVegas.keys():
    len_i.append(len(IuVegas[k]))
for k in UiVegas.keys():
    len_u.append(len(UiVegas[k]))

In [ ]:
len_u.sort()
len_u = len_u[:int(len(len_u)*0.999)]

In [ ]:
len_i.sort()
len_i = len_i[:int(len(len_i)*0.999)]

In [ ]:
import matplotlib.pyplot as plt

plt.hist(len_i,bins=100)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.hist(len_u,bins=6000)
plt.show()

In [ ]:
Lambda = 0
Alpha = 10**-4
K = 1

In [ ]:
Gamma_u = {}
Gamma_r = {}

In [ ]:
for user in VegasUser:
    Gamma_u[user] = np.random.rand(K)
for rest in VegasRest:
    Gamma_r[rest] = np.random.rand(K)

In [ ]:
def logit(t):
    return np.exp(-t)/(1+np.exp(-t))

In [ ]:
def sigmoid(t):
    return 1-logit(t)

In [ ]:
import math
def vlen(t):
    su = 0
    for i in t:
        su += i*i
    return su

In [ ]:
CUser = []
Cirest = []
Cjrest = []
iterations = 1000000
l=[]

In [ ]:
def generate_validation_set(VegasUser, VegasRest):
    #to be used every 10k iterations
    triple_arr = []
    for i in range(1000):
        user = np.random.choice(VegasUser)  #pick user
        irest = np.random.choice(IuVegas[user])  #pick item i
        jrest = np.random.choice(list(set(VegasRest) - set(IuVegas[user])))  #pick item j
        triple_arr.append([user,irest,jrest])
    return triple_arr

In [ ]:
val_array = generate_validation_set(VegasUser,VegasRest)

In [ ]:
len(val_array)

In [ ]:
def val_loss():
    su = 0
    for i in range(len(val_array)):
        user = val_array[i][0]
        irest = val_array[i][1]
        jrest = val_array[i][2]
        xuij = np.dot(Gamma_u[user],Gamma_r[irest]-Gamma_r[jrest])
        su += np.log(sigmoid(xuij))
    return su

In [ ]:
def loss(CUser,Cirest,Cjrest):
    su = 0
    for i in range(len(CUser)):
        user = CUser[i]
        irest = Cirest[i]
        jrest = Cjrest[i]
        xuij = np.dot(Gamma_u[user],Gamma_r[irest]-Gamma_r[jrest])
        su += np.log(sigmoid(xuij))
    for user in VegasUser:
        su -= Lambda* vlen(Gamma_u[user]) * vlen(Gamma_u[user])
    for rest in VegasRest:
        su -= Lambda* vlen(Gamma_r[rest]) * vlen(Gamma_r[rest])
    return su

In [ ]:
# for i in range(iterations):
#     user = random.choice(Iu.keys())
#     CUser.append(user)
#     irest = random.choice(Iu[user])
#     Cirest.append(irest)
#     jrest = random.choice(list(AllRestSet - set(Iu[user])))
#     Cjrest.append(jrest)
# for i in range(iterations):
#     user = CUser[i]
#     irest = Cirest[i]
#     jrest = Cjrest[i]
#     xuij = np.dot(Gamma_u[user],Gamma_r[irest]-Gamma_r[jrest])
#     Gamma_u[user] += Alpha*(logit(xuij)*(Gamma_r[irest]-Gamma_r[jrest]) + Lambda*Gamma_u[user])
#     Gamma_r[irest]+= Alpha*(logit(xuij)*(Gamma_u[user]) + Lambda*Gamma_r[irest])
#     Gamma_r[jrest]+= Alpha*(logit(xuij)*(-Gamma_u[user]) + Lambda*Gamma_r[jrest])
#     l.append(loss(CUser,Cirest,Cjrest))

In [ ]:
# plt.plot(range(iterations),l)
# plt.show()

In [ ]:
train_l = []
val_l = []

In [ ]:
S_uj_train = {}

for i in range(iterations):
    user = random.choice(VegasUser)
    CUser.append(user)
    irest = random.choice(IuVegas[user])
    Cirest.append(irest)
    jrest = random.choice(list(set(VegasRest) - set(IuVegas[user])))
    Cjrest.append(jrest)
for i in range(iterations):
    user = CUser[i]
    irest = Cirest[i]
    jrest = Cjrest[i]
    xuij = np.dot(Gamma_u[user],Gamma_r[irest]-Gamma_r[jrest])
    x= Alpha*(logit(xuij)*(Gamma_r[irest]-Gamma_r[jrest]))
    y = Alpha*(logit(xuij)*(Gamma_u[user]))
    z = Alpha*(logit(xuij)*(-Gamma_u[user]))
    for user in VegasUser:
        Gamma_u[user] += Alpha*Lambda*Gamma_u[user]
    for rest in VegasRest:
        Gamma_r[rest] += Alpha*Lambda*Gamma_r[rest]
    Gamma_u[user] += x
    Gamma_r[irest] += y
    Gamma_r[jrest] += z
    if i%100== 0:
        train_l.append(loss(CUser,Cirest,Cjrest))
        val_l.append(val_loss())
        print(i/100,val_l[-1],train_l[-1])

In [ ]:
iterations = 5000000

In [ ]:
for i in range(iterations):
    user = random.choice(VegasUser)
    CUser.append(user)
    irest = random.choice(IuVegas[user])
    Cirest.append(irest)
    jrest = random.choice(list(set(VegasRest) - set(IuVegas[user])))
    Cjrest.append(jrest)
    
    if not S_uj_train.has_key(user):
        S_uj_train[user] = [jrest]
    if jrest not in S_uj_train[user]:
        S_uj_train[user].append(jrest)
    if irest not in S_uj_train[user]:
        S_uj_train[user].append(irest)
    
    if i%10000==0:
        print i

In [ ]:
with open('CUser.txt', 'w') as outfile:
    json.dump(CUser, outfile)

In [ ]:
with open('Cirest.txt', 'w') as outfile:
    json.dump(Cirest, outfile)

In [ ]:
with open('Cjrest.txt', 'w') as outfile:
    json.dump(Cjrest, outfile)

In [ ]:
with open('CUser.txt', 'r') as outfile:
    CUser = json.load(outfile)

In [ ]:
with open('Cirest.txt', 'r') as outfile:
    Cirest = json.load(outfile)

In [ ]:
with open('Cjrest.txt', 'r') as outfile:
    Cjrest = json.load(outfile)

In [ ]:
len(Cjrest)

In [ ]:
iterations = 10**8

In [ ]:
for i in range(iterations):
#     j= random.randint(0,9999)
#     user = random.choice(VegasUser)
#     CUser[j] = user
#     irest = random.choice(IuVegas[user])
#     Cirest[j] = irest
#     jrest = random.choice(list(set(VegasRest) - set(IuVegas[user])))
#     Cjrest[j] = jrest
    if i >=  5000000:
        j =np.random.randint(0,5000000)
        user = CUser[j]
        irest = Cirest[j]
        jrest = Cjrest[j]
    else:
        user = CUser[i]
        irest = Cirest[i]
        jrest = Cjrest[i]

    xuij = np.dot(Gamma_u[user],Gamma_r[irest]-Gamma_r[jrest])
    x= Alpha*(logit(xuij)*(Gamma_r[irest]-Gamma_r[jrest]))
    y = Alpha*(logit(xuij)*(Gamma_u[user]))
    z = Alpha*(logit(xuij)*(-Gamma_u[user]))
    
#     for user in VegasUser:
#         Gamma_u[user] += Alpha*Lambda*Gamma_u[user]
#     for rest in VegasRest:
#         Gamma_r[rest] += Alpha*Lambda*Gamma_r[rest]
    Gamma_u[user] += x
    Gamma_r[irest] += y
    Gamma_r[jrest] += z
    if i%(10**5)== 0:
        #train_l.append(loss(CUser,Cirest,Cjrest))
        #val_l.append(val_loss())
        print(i/10**5) #,val_l[-1])#,train_l[-1])

In [ ]:
len(val_l)

In [ ]:
plt.plot(range(50),val_l)
plt.show()

In [ ]:
plt.plot(range(100),train_l)
plt.show()

In [ ]:
total = 0
count = 0
for user in VegasUser:
    count += 1
    print count
    su = 0
    i = S_ui_test[user]
    jAbsent = []
    if S_uj_train.has_key(user):
        jAbsent = list(set(VegasRest) - set(S_uj_train[user]) - set(S_ui_test))
    else:
        jAbsent = list(set(VegasRest) - set(S_ui_test))
    
    for j in jAbsent:
        if np.dot(Gamma_u[user],Gamma_r[i]) > np.dot(Gamma_u[user],Gamma_r[j]):
            su += 1
    total += su*1.0/(len(jAbsent))
total = total*1.0/(len(VegasUser))
print total

0.01, Sai's method, 10**-4, 10000 iter, K = 10, AUC = 0.515965891654

0, 0.84067086067, 10**-4, 10**8, K = 10
0, 0.84409049708, 10**-4, 2*10**8, K = 10
0,0.84474016538,10**-4,3*10**8,k=10
0,0.84431301534,3X10**-4
0,0.839135075941
0,0.842078864994,10**-4,10**8,k=16
0,0.843955989018,10**-4,2*10**8,k=16,,
0,0.8442609638, 10**-4,3*10**8,k=16
0.840741215544, k = 8
0.83630605302, k = 4
0.831370757313, k = 2
0.820573011204, k = 1


### Baseline

In [ ]:
UiVegas_b = readJSONfromFile('yelp_user_UiVegas.json') 
UiVegas_b = UiVegas_b[0]

In [ ]:
Ui_len= {}
for rest in UiVegas_b:
    Ui_len[rest]= len(UiVegas_b[rest])


In [ ]:
total = 0
count = 0
for user in VegasUser:
    #count += 1
    #print count
    su = 0
    i = S_ui_test[user]
    xui = Ui_len[i]
    jAbsent = []
    if S_uj_train.has_key(user):
        jAbsent = list(set(VegasRest) - set(S_uj_train[user]) - set(S_ui_test))
    else:
        jAbsent = list(set(VegasRest) - set(S_ui_test))
    
    for j in jAbsent:
        xuj = Ui_len[j]
        if xui > xuj :
            su += 1
    total += su*1.0/(len(jAbsent))
total = total*1.0/(len(VegasUser))


In [ ]:
print("Baseline AUC ={}".format(total))